In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import talib as ta

import numpy as np
import pandas as pd

from quantopian.pipeline.filters.morningstar import Q500US, Q1500US, Q3000US
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

from quantopian.pipeline.data import EquityPricing
from quantopian.pipeline.data import morningstar

from quantopian.pipeline.factors import PercentChange, RSI, SimpleMovingAverage, SimpleBeta, Returns

from quantopian.pipeline.data.factset import Fundamentals

from quantopian.pipeline.factors import CustomFactor
#from quantopian.pipeline.data.quandl import cboe_vix as vixData

In [ ]:
universe = Q500US()
start_date = '2020-01-01'
end_date='2020-10-24'

In [ ]:
class getVix(CustomFactor):  
    def compute(self, today, assets, out, vix):   
        out[:] = vix[-1]
        
class pathquality(CustomFactor):  
    def compute(self, today, assets, out, values):  
        out[:] = np.nanstd(np.diff(values, axis=0), axis = 0)

In [ ]:
def make_pipeline():
    price = EquityPricing.close.latest
    close = EquityPricing.close
    
    EMA200 = SimpleMovingAverage(inputs=[close], window_length=200)/price*100
    delta125 = PercentChange(inputs=[close], window_length=125)*100
    EMA50 = SimpleMovingAverage(inputs=[close], window_length=50)/price*100
    delta20 = PercentChange(inputs=[close], window_length=20)*100
    RSI14 = RSI(inputs=[close], window_length=15)
    
    
    SCTR = 0.3*EMA200 + 0.3*delta125 + 0.15*EMA50 + 0.15*delta20 + 0.05*RSI14
    
    beta = SimpleBeta(symbols('SPY') , regression_length = 252)
    
    #vix  = getVix(inputs  = [vixData.vix_close],window_length = 1)
    
    lastyearsreturns = Returns(window_length = 252)
    
    dailyreturns = Returns(window_length = 2)
    quality = pathquality(inputs = [close], window_length = 252)  
    
    classification = morningstar.asset_classification.morningstar_industry_group_code.latest
    
    return Pipeline(
        columns={
            'prev_close':price,
            'Stock Ranking':SCTR,
            'classification':classification,
            'volatility': quality,
            'beta':beta,
            #'vix': vix
        },
        screen=universe
    )


In [ ]:
pipe = make_pipeline()
data = run_pipeline(pipe, start_date, end_date)
pricing_data = get_pricing(data.index.levels[1], start_date, end_date, fields='close_price')


In [ ]:
reindex = data.reorder_levels([1, 0]).reset_index().set_index('level_0')

In [ ]:
NVAX = reindex.loc[symbols('NVAX'),:]

In [ ]:
NVAX = NVAX.set_index('level_1')

In [ ]:
NVAX['Stock Ranking'].plot()